<a href="https://colab.research.google.com/github/Eminent01/Text-Classification-using-Fully-connected-FC-NN/blob/main/Text_Classification_FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter

from pyparsing import WordStart
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import requests
from io import StringIO
import math
import gensim.downloader as api
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from gensim.models import Word2Vec

class PreprocessData:
    def __init__(self,path,lang='english'): 
        self.dataset=pd.read_csv(path, sep=",")
        self.dataset=self.dataset[["content","score"]]
        self.stopwords=stopwords.words(lang)
        self.preprocess()
    
    @staticmethod
    def remove_punctuation(text):
        '''a function for removing punctuation'''
        # replacing the punctuations with no space, 
        # which in effect deletes the punctuation marks 
        translator = str.maketrans('', '', string.punctuation)
        # return the text stripped of punctuation marks
        return text.translate(translator)

    #A function to remove the stopwords
    def remove_stopwords(self,text):
        text = [word.lower() for word in text.split() if word.lower() not in self.stopwords]
        # joining the list of words with space separator
        return " ".join(text)

    def preprocess(self):
        self.dataset.iloc[:,0] = self.dataset.iloc[:,0].apply(self.remove_punctuation)
        self.dataset.iloc[:,0] = self.dataset.iloc[:,0].apply(self.remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path="/content/drive/MyDrive/Text_classification_FCN/reviews.csv"
dataset=PreprocessData(path)

In [4]:
dataset.dataset.head()

,content,score
0,cannot open app anymore,1
1,begging refund app month nobody replying,1
2,costly premium version approx indian rupees 91...,1
3,used keep organized 2020 updates made mess thi...,1
4,dan birthday oct 28,1


In [ ]:
# wv = api.load('glove-wiki-gigaword-50')

In [ ]:
# from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
# import gensim.models

# # This should take approximately 3 minutes
# model = gensim.models.Word2Vec(sentences=sentences,min_count=1)

In [ ]:
# print(list(model.wv.vocab))

In [6]:
from collections import Counter

from pyparsing import WordStart
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import requests
from io import StringIO
import math
import gensim.downloader as api
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
from gensim.models import Word2Vec


class WordRepresentation:
    def __init__(self,dataset,method="word2vec",sentence_length=20,vector_length=50):
        if method not in ["word2vec","glove"]:
            print("Give 'word2vec' or 'glove' method")
            return None
        self.dataset=dataset
        self.glove={}
        self.word2vec={}
        self.sentence_length=sentence_length
        self.vector_length=vector_length
        self.method=method
        if method=="word2vec":
            self.compute_word2vec()
        else:
            self.compute_glove()
        self.final_data=[]
        self.represent_all_sentences()
    
    #Compute Glove
    def compute_glove(self):
      wv = api.load('glove-wiki-gigaword-50')
      #self.glove["nan"]=(np.zeros(50,),0)
      #word_index=1
      for i in range(len(self.dataset)):
        document=self.dataset.iloc[i,0]
        word_list=document.split(" ")
        for word in word_list:
          if word in wv.vocab:
            self.glove[word]=wv.word_vec(word)
            # self.glove[word]=(wv.word_vec(word),word_index)
            # word_index+=1
    
    @staticmethod
    def tokenize(data): 
      word = []
      for i in range(len(data)):
        new_doc = data[i].split()
        word.append(new_doc)
      return word
    
    def compute_word2vec(self):
      token = WordRepresentation.tokenize(self.dataset['content'])
      model = Word2Vec(sentences=token, workers = 1, size = 50, min_count = 1, window = 3)
      # self.word2vec["nan"]=(np.zeros(50,),0)d
      # word_index=1
      for i in range(len(self.dataset)):
        document=self.dataset.iloc[i,0]
        word_list=document.split(" ")
        for word in word_list:
          if word in model.wv.vocab:
            self.word2vec[word]=model.wv.word_vec(word)
            # self.word2vec[word]=(model.wv.word_vec(word),word_index)
            # word_index+=1
    
    def sentence_representation(self,sentence):
        the_sentence=sentence.split(' ')
        if self.method=="word2vec":
          words_dict=self.word2vec
        else:
          words_dict=self.glove
        
        matrix=np.zeros((self.sentence_length,self.vector_length))
        i =0
        for word in the_sentence:
          if word in words_dict:
            matrix[i]=words_dict[word]
            i+=1
          if i>=self.sentence_length:
            break
        return matrix
        
    def represent_all_sentences(self):
      for i in range(len(self.dataset)):
        document=self.dataset.iloc[i,0]
        y=torch.tensor([self.dataset.iloc[i,1]])
        x_w2v=self.sentence_representation(document)
        y_w2v=torch.tensor(x_w2v.reshape(1,-1))
        y_w2v=y_w2v.float()
        self.final_data.append((y_w2v,y))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
max_sentence_length=0
for i in range(len(dataset.dataset)):
  doc=dataset.dataset.iloc[i,0]
  list_words=doc.split(" ")
  # print(len(list_words))
  number_of_words=len(list_words)
  if number_of_words>max_sentence_length:
    max_sentence_length=number_of_words  

In [37]:
new_all_data=WordRepresentation(dataset.dataset,sentence_length=max_sentence_length, method= 'glove')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [14]:
max_sentence_length

356

In [15]:
# new_data.represent_all_sentences()

In [23]:
new_all_data.final_data

[(tensor([[-0.7822,  1.0833,  0.4661,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.0625,  0.1035,  0.0088,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.0757,  0.1102,  0.0067,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.6658,  1.6293, -0.4255,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.1066,  0.1800,  0.0547,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.4810,  0.7859,  0.2341,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.9472,  1.3690,  0.1108,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.3815,  0.5624,  0.2628,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.5352,  0.8669,  0.3066,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.0598,  0.0848,  0.0164,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([[-0.1990,  0.2861,  0.1084,  ...,  0.0000,  0.0000,  0.0000]]),
  tensor([1])),
 (tensor([

In [ ]:
# new_data.glove

In [ ]:
# new_data.word2vec

In [24]:
len(new_all_data.word2vec)

13948

In [19]:
len(new_all_data.glove)

0

In [38]:
input_size=len(new_all_data.glove)*20

In [39]:
vector_lenght=50
sentence_lenght = 356
input_dim=vector_lenght*sentence_lenght
num_classes=5

In [25]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [26]:
device

'cpu'

In [27]:
class FcNeuralNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(FcNeuralNet, self).__init__()
        self.fc1= nn.Linear(input_dim, hidden_dim)
        self.fc2= nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
      """
      The forward pass of the fully connected layer
      """
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      out=F.log_softmax(x, dim =1)
      return out

In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [29]:
model1=FcNeuralNet(10,10,5)

In [31]:
x=new_all_data.final_data[3]

In [32]:
model2=FcNeuralNet(input_dim,70,5)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4, weight_decay= 1e-3) 

### Split for glove

In [40]:
from sklearn.model_selection import train_test_split
glove_train, glove_test = train_test_split(new_all_data.final_data, test_size = 0.2, random_state = 42, shuffle = True)

### Split for word2vec

In [35]:
from sklearn.model_selection import train_test_split
w2v_train, w2v_test = train_test_split(new_all_data.w2v_final_data, test_size = 0.2, random_state = 42, shuffle = True)

AttributeError: ignored

In [41]:
len(glove_train)

9996

In [42]:
len(w2v_train)

NameError: ignored

In [43]:
num_epochs = 10

### Train for Glove

In [44]:
glosses = []
for epoch in range(num_epochs):
  model2.train()
  running_loss = 0.0
  for i in glove_train:
    sentences, labels = i

    # Move tensors to the configured device
    # sentences = sentences.to(device)
    # labels = labels.to(device)
        
    # Forward pass
    outputs = model2(sentences)
    glove_loss = criterion(outputs, labels-1)    
    # Backprpagation and optimization
    glove_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    running_loss += glove_loss.item() 
            
  loss_values = (running_loss / len(glove_train))
  glosses.append(loss_values)
        
        
  print("=" * 25)
  print('| Epoch {:3d}| Loss: {:.4f}'.format(epoch+1,glove_loss.item()))

| Epoch   1| Loss: 1.3790
| Epoch   2| Loss: 1.2318
| Epoch   3| Loss: 1.0877
| Epoch   4| Loss: 0.9725
| Epoch   5| Loss: 0.8695
| Epoch   6| Loss: 0.7994
| Epoch   7| Loss: 0.7243
| Epoch   8| Loss: 0.5528
| Epoch   9| Loss: 0.4805


KeyboardInterrupt: ignored

### train for word2vec

In [ ]:
wlosses = []

for epoch in range(num_epochs):
  model2.train()
  running_loss = 0.0
  for i in w2v_train:
    sentences, labels = i

    # Move tensors to the configured device
    # sentences = sentences.to(device)
    # labels = labels.to(device)
  
    # Forward pass
    outputs = model2(sentences)
    w2vec_loss = criterion(outputs, labels-1)    
    
    # Backprpagation and optimization
    w2vec_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    running_loss += w2vec_loss.item()
            
  loss_values = (running_loss / len(w2v_train))
  wlosses.append(loss_values)    
  print("=" * 25)
  print('| Epoch {:3d}| Loss: {:.4f}'.format(epoch+1,w2vec_loss.item()))  

| Epoch   1| Loss: 1.5392
| Epoch   2| Loss: 1.5395
| Epoch   3| Loss: 1.5422
| Epoch   4| Loss: 1.5453
| Epoch   5| Loss: 1.5401
| Epoch   6| Loss: 1.5387
| Epoch   7| Loss: 1.5378
| Epoch   8| Loss: 1.5367
| Epoch   9| Loss: 1.5342
| Epoch  10| Loss: 1.5350


## Predict

### for glove

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for i in glove_test:
      sentences, labels = i
      outputs = model2(sentences)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted+1 == labels).sum().item()
      print(predicted,labels)
    
print("="* 33)
print('Accuracy test: {:.2f} %'.format(100 * correct / total))
print("="* 33)

tensor([4]) tensor([5])
tensor([0]) tensor([1])
tensor([0]) tensor([1])
tensor([1]) tensor([2])
tensor([0]) tensor([1])
tensor([4]) tensor([5])
tensor([0]) tensor([1])
tensor([4]) tensor([5])
tensor([4]) tensor([5])
tensor([1]) tensor([2])
tensor([4]) tensor([5])
tensor([0]) tensor([5])
tensor([4]) tensor([5])
tensor([4]) tensor([2])
tensor([1]) tensor([4])
tensor([4]) tensor([1])
tensor([4]) tensor([5])
tensor([3]) tensor([5])
tensor([4]) tensor([5])
tensor([3]) tensor([3])
tensor([0]) tensor([2])
tensor([4]) tensor([3])
tensor([4]) tensor([4])
tensor([4]) tensor([5])
tensor([4]) tensor([4])
tensor([3]) tensor([3])
tensor([0]) tensor([1])
tensor([4]) tensor([1])
tensor([0]) tensor([3])
tensor([0]) tensor([3])
tensor([0]) tensor([4])
tensor([2]) tensor([3])
tensor([4]) tensor([1])
tensor([4]) tensor([5])
tensor([4]) tensor([5])
tensor([3]) tensor([4])
tensor([1]) tensor([2])
tensor([2]) tensor([1])
tensor([1]) tensor([3])
tensor([3]) tensor([1])
tensor([3]) tensor([5])
tensor([2]) tens

## Test real data with glove
We can now test out function with real word sentences

### for word2vec

In [ ]:
with torch.no_grad():
  correct = 0
  total = 0
  for i in w2v_test:
    sentences, labels = i

    outputs = model2(sentences)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted+1 == labels).sum().item()
    print(predicted,labels)
    
print("="* 33)
print('Accuracy test : {:.2f} %'.format(100 * correct / total))
print("="* 33)

tensor([3]) tensor([5])
tensor([3]) tensor([1])
tensor([0]) tensor([1])
tensor([0]) tensor([2])
tensor([1]) tensor([1])
tensor([4]) tensor([5])
tensor([4]) tensor([1])
tensor([4]) tensor([5])
tensor([4]) tensor([5])
tensor([1]) tensor([2])
tensor([4]) tensor([5])
tensor([4]) tensor([5])
tensor([4]) tensor([5])
tensor([4]) tensor([2])
tensor([0]) tensor([4])
tensor([4]) tensor([1])
tensor([4]) tensor([5])
tensor([3]) tensor([5])
tensor([4]) tensor([5])
tensor([1]) tensor([3])
tensor([0]) tensor([2])
tensor([4]) tensor([3])
tensor([1]) tensor([4])
tensor([1]) tensor([5])
tensor([4]) tensor([4])
tensor([3]) tensor([3])
tensor([0]) tensor([1])
tensor([4]) tensor([1])
tensor([0]) tensor([3])
tensor([0]) tensor([3])
tensor([0]) tensor([4])
tensor([1]) tensor([3])
tensor([4]) tensor([1])
tensor([4]) tensor([5])
tensor([1]) tensor([5])
tensor([3]) tensor([4])
tensor([0]) tensor([2])
tensor([0]) tensor([1])
tensor([1]) tensor([3])
tensor([0]) tensor([1])
tensor([4]) tensor([5])
tensor([1]) tens

## Visualisation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot(cost1, cost2):
  plt.title("loss Glove vs Word2Vec")
  plt.xlabel('Epochs')
  plt.ylabel('Cost')
  plt.plot(cost1, "r",  label= 'Glove Loss')
  plt.plot(cost2, "b",  label= 'Word2Vec Loss')
  plt.legend()
  plt.show()

In [ ]:
plot(glosses, wlosses)